In [1]:
import tensorflow as tf

In [125]:
data_path = 'data/'

train_csv_file_path = data_path + 'train_ship_segmentations_v2.csv'
train_image_path = data_path + 'train_v2/'

In [126]:
def load_csv(path):
    return tf.data.experimental.make_csv_dataset(
        train_csv_file_path,
        batch_size=32, # required
        column_names=['ImageId', 'EncodedPixels'],
    )

train_csv = load_csv(train_csv_file_path)

In [118]:
for item in train_csv.unbatch().take(1):
    print(item)
    print(item['ImageId'])
    print(item['EncodedPixels'])

OrderedDict([('ImageId', <tf.Tensor: shape=(), dtype=string, numpy=b'0ab3d7bbc.jpg'>), ('EncodedPixels', <tf.Tensor: shape=(), dtype=string, numpy=b''>)])
tf.Tensor(b'0ab3d7bbc.jpg', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)


In [127]:
def process_img(file):
    img = tf.io.read_file(train_image_path + file)
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def process_label(label):
    return label

def process_batch(csv_item):
    X = process_img(csv_item['ImageId'])
    y = process_label(csv_item['EncodedPixels'])
    return X, y

m = train_csv.unbatch().map(process_batch)

for item in m.take(1):
    print(item)

(<tf.Tensor: shape=(768, 768, 3), dtype=uint8, numpy=
array([[[27, 44, 51],
        [28, 45, 52],
        [27, 44, 51],
        ...,
        [29, 47, 51],
        [29, 47, 51],
        [29, 47, 51]],

       [[32, 49, 56],
        [32, 49, 56],
        [30, 47, 54],
        ...,
        [26, 44, 48],
        [27, 45, 49],
        [27, 45, 49]],

       [[33, 50, 57],
        [33, 50, 57],
        [32, 49, 56],
        ...,
        [25, 43, 47],
        [26, 44, 48],
        [27, 45, 49]],

       ...,

       [[22, 46, 50],
        [21, 45, 49],
        [24, 45, 50],
        ...,
        [29, 48, 55],
        [26, 45, 52],
        [24, 43, 50]],

       [[23, 47, 51],
        [23, 47, 51],
        [27, 48, 53],
        ...,
        [28, 46, 56],
        [26, 44, 54],
        [24, 42, 52]],

       [[23, 47, 51],
        [23, 47, 51],
        [28, 49, 54],
        ...,
        [28, 46, 56],
        [25, 43, 53],
        [23, 41, 51]]], dtype=uint8)>, <tf.Tensor: shape=(), dtype=string, 